# CVNN Playground

The purpose of this notebook is to provide a simple playground for the Complex-Valued Neural Networks (CVNN) implementation, in the aim of proving their strengths vs real-valued neural networks. The CVNN is a neural network that uses complex numbers as input, output, and weights. The CVNN is implemented using PyTorch and the complextorch library.  
  
The notebook calls to a couple of other python files:  
- `cvnn_data.py` contains the data generation functions.  
- `cvnn_model.py` contains the CVNN model definitions.  

The notebook is divided into the following sections:
1. **Imports and settings** - Importing the necessary libraries and setting the data paths etc.
2. **Data Preparation** - Generating the data for the CVNN. By default this adds noise to the data, randomly phase shifts the data by either -pi, 0, or pi, and randomly shifts the signal in time by a value between -0.1s and 0.1s.
3. **Training loop definition**.
4. **Complex Valued Training** - Training `ComplexValuedNN` defined in `cvnn_models.py` on the generated data. This model uses two 1d complex convolutional layers with fully complex activation functions (Cardoid) and two fully connected layers.  
5. **Real Valued Training** - Training `RealValuedNN` defined in `cvnn_models.py` on the generated data. This model uses two 1d real convolutional layers (real and imag in a channel each) with ReLU activation functions and two fully connected layers.

### Results:  
Conv layers are required? 
It seems CVNNs can learn better than RVNNs when turning the number of conv filters down so the network goes <1m parameters.  

Plot of the loss and accuracy of the CVNN and RVNN models during training:
![lossoutput_1m_pars.png](../cvnn_lossoutput_1m_pars.png)

## Imports and settings

In [ ]:
import numpy as np
from numpy.fft import rfft, irfft

from functools import partial
rfft = partial(rfft, norm='ortho')
irfft = partial(irfft, norm='ortho')
import matplotlib.pyplot as plt

import torch
import torch
import torch.nn as nn
from torch.utils.data import Sampler
from typing import Iterator, Sized, List
from torch.utils.data import DataLoader
import torch
from cvnn_models import ComplexValuedNN, RealValuedNN
from cvnn_data import get_data, GlitchDataset

import matplotlib.pyplot as plt
import os
import random
from antiglitch import SnippetNormed

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [ ]:
# Torch setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

torch.backends.cuda.matmul.allow_tf32 = True

def memsum():
    """Prints a summary of the GPU memory usage."""
    print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
# switches to control training
train_complex, test_complex = True, True
train_real, test_real = True, True
train_real_td, test_real_td = False, False

# define dataset
NTRAIN = 100000
NTEST = 50000
NBATCH = 4096

rootdir = '/home/xangma/OneDrive/repos/antiglitch/'
datadir = rootdir + 'data/'

# set seed for data generation - noise, augmentation, etc.
np.random.seed(0)

## Data preparation

In [ ]:
# get data from datadir
distributions, glitches, ifos, ml_models = get_data(datadir)
print(f"Loaded {len(distributions)} distributions, {len(glitches)} glitches, {ifos} ifos, and {ml_models} ml_models.")

In [ ]:
# create training and test datasets
train_data = GlitchDataset(datadir, ifos, ml_models, glitches, distributions, NTRAIN, NTEST, device, True, True, True, 'train', 'complex', None)
test_data = GlitchDataset(datadir, ifos, ml_models, glitches, distributions, NTRAIN, NTEST, device, True, True, True, 'test', 'complex', train_data)

# # print max and mins of training and test data
print(f"Train data real + imag maxs: {torch.max(train_data.x_arr.real)}, {torch.max(train_data.x_arr.imag)}")
print(f"Test data real + imag maxs: {torch.max(test_data.x_arr.real)}, {torch.max(test_data.x_arr.imag)}")
print(f"Train data real + imag mins: {torch.min(train_data.x_arr.real)}, {torch.min(train_data.x_arr.imag)}")
print(f"Test data real + imag mins: {torch.min(test_data.x_arr.real)}, {torch.min(test_data.x_arr.imag)}")


In [ ]:
# Plot some data

trainitem = train_data.__getitem__(0)
testitem = test_data.__getitem__(0)
trainitem=trainitem[0].cpu().numpy()
testitem=testitem[0].cpu().numpy()

# Get a real snippet
ifo = np.random.choice(ifos)
ml_model = np.random.choice(ml_models)
glitch_num = random.choice(glitches[ifo][ml_model])
glitch_num = glitch_num['num']
# create a SnippetNormed object
snip = SnippetNormed(ifo, ml_model, glitch_num, datadir)
inf = {}
inf['freqs'] = np.linspace(0, 4096, 513)
snip.set_infer(inf)
# get the glitch data in the frequency domain
actual_item = rfft(snip.whts)
# scale it by the training scalings
actual_item_real = (actual_item.real - train_data.tr_x_mean_real) / train_data.tr_x_std_real
actual_item_imag = (actual_item.imag - train_data.tr_x_mean_imag) / train_data.tr_x_std_imag
actual_item = actual_item_real + 1j * actual_item_imag
actual_item = np.abs(actual_item)
# plot them all side by side
plt.figure()
plt.subplot(1,3,1)
plt.loglog(np.abs(trainitem))
plt.title('Train')
plt.subplot(1,3,2)
plt.loglog(np.abs(testitem))
plt.title('Test')
plt.subplot(1,3,3)
plt.loglog(actual_item)
plt.title('Actual')
plt.show()

In [ ]:
class FastRandomSampler(Sampler[int]):
    def __init__(self, data_source: Sized, batch_size: int, generator=None) -> None:
        self.data_source = data_source
        self.data_len = len(self.data_source)
        self.batch_size = batch_size
        self.generator = generator or torch.Generator()
        self.epoch_indices = []
        self.set_epoch(0)  # Shuffle at initialization

    def _shuffle_indices(self):
        self.epoch_indices = torch.randperm(self.data_len, generator=self.generator).tolist()

    def __iter__(self) -> Iterator[List[int]]:
        # Yield batches of indices using slicing
        return (self.epoch_indices[i:i + self.batch_size] for i in range(0, self.data_len, self.batch_size))

    def __len__(self) -> int:
        # Number of batches in an epoch
        return (len(self.data_source) + self.batch_size - 1) // self.batch_size
    
    def set_epoch(self, epoch: int) -> None:
        self._shuffle_indices()  # Shuffle indices for the new epoch

In [ ]:
# Load into DataLoader
tr_sampler = FastRandomSampler(train_data, NBATCH)
te_sampler = FastRandomSampler(test_data, NBATCH)
collate_fn = lambda x: tuple(x)
train_loader = DataLoader(dataset=train_data, sampler=tr_sampler,   drop_last=False)
test_loader = DataLoader(dataset=test_data, sampler=te_sampler,   drop_last=False)
list(train_loader.__iter__())[0][0].shape

## Training Loop Definition

In [ ]:
# Training loop
def train(model, criterion, optimizer, scheduler, train_loader, test_loader, num_epochs, model_path, writer):
    epoch_count=0
    # Load model if it exists
    try:
        # find model with highest epoch number
        modelfn = model_path.split('/')[-1]
        modeldir = model_path.split('/')[:-1].join('/')
        model_files = os.listdir(modeldir)
        model_files = [f for f in model_files if f.endswith('.pt') and f.startswith(modelfn.split('.')[0])]
        if len(model_files) > 0:
            model_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
            model_path = model_path + model_files[-1]
            print(f'Loading model {model_path}')
            model.load_state_dict(torch.load(model_path))
            print('Model loaded')
            epoch_count = int(model_path.split('_')[-1].split('.')[0])
    except:
        print('Model not loaded')
    torch.autograd.set_detect_anomaly(True)
    loss_arr = []
    val_loss_arr = []
    while epoch_count < num_epochs:
        model.train()
        total_training_loss = 0
        for i, data in enumerate(train_loader):
            inputs, targets = data
            # Forward pass
            outputs = model(inputs[0])
            loss = criterion(outputs, targets[0])
            total_training_loss += loss.item()
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_arr.append(total_training_loss/len(train_loader))

        # Test the model
        model.eval()
        with torch.no_grad():
            total_loss = 0
            for i, data in enumerate(test_loader):
                inputs, targets = data
                outputs = model(inputs[0])
                val_loss = criterion(outputs, targets[0]).item()
                total_loss += val_loss
            val_loss_arr.append(total_loss/len(test_loader))
        print(f"Epoch [{epoch_count+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {total_loss/len(test_loader):.4f}")
        
        # update the learning rate
        scheduler.step(total_loss/len(test_loader))

        train_loader.sampler.set_epoch(epoch_count)
        epoch_count += 1
        
        writer.add_scalars("Loss", {"Train": total_training_loss/len(train_loader), "Validation": total_loss/len(test_loader)}, epoch_count)
        writer.add_scalar("Learning rate", optimizer.param_groups[0]['lr'], epoch_count)
        
        # save the model every 10 epochs
        if epoch_count % 10 == 0 and epoch_count != 0:
            torch.save(model.state_dict(), model_path.split('.')[0] + f'_{epoch_count}.pt')
        writer.flush()
    return loss_arr, val_loss_arr
            


## Complex Valued Training

In [ ]:
if train_complex:
    if any([x in locals() for x in ['model', 'criterion', 'optimizer', 'scheduler', 'writer']]):
        del model, criterion, optimizer, scheduler, writer
        torch.cuda.empty_cache()
    # Create an instance of the network
    def get_complex_model():
        return ComplexValuedNN(n_conv_layers=2, conv_filters=[4,8], conv_kernel_size=[1,3], n_fc_layers=1, n_fc_units=128)
    
    model = get_complex_model()
    # put model on device
    model.to(device)

    model = torch.compile(model)
    # Print the model architecture
    print(model)
    print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)
    writer = SummaryWriter("Complex_valued")
    model_path = rootdir + 'notebooks/complex_nn_model.pt'

In [ ]:
torch.cuda.empty_cache()
if train_complex:
    complex_train_loss_arr,  complex_val_loss_arr = train(model, criterion, optimizer, scheduler, train_loader, test_loader, 100, model_path, writer)
    # save the loss arrays into a single file
    np.savez(rootdir + 'notebooks/complex_loss_arrays.npz', train=complex_train_loss_arr, val=complex_val_loss_arr)
    del model, criterion, optimizer, scheduler, writer
    torch.cuda.empty_cache()
    memsum()

In [ ]:
if test_complex:
    # Predictions from Complex model
    # Load model
    model = get_complex_model()
    model = torch.compile(model)
    
    modelfn = model_path.split('/')[-1]
    modeldir = '/'.join(model_path.split('/')[:-1])
    model_files = os.listdir(modeldir)
    model_files = [f for f in model_files if f.endswith('.pt') and f.startswith(modelfn.split('.')[0])]
    if len(model_files) > 0:
        model_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
        model_path = modeldir + '/' + model_files[-1]
    
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()


In [ ]:
if test_complex:
    # Get 10 inputs
    inputs, targets = next(iter(test_loader))
    outputs = model(inputs[0])

    if type(outputs) == torch.Tensor:
        outputs = outputs.cpu().detach().numpy()
    if type(targets) == torch.Tensor:
        targets = targets[0].cpu().detach().numpy()
    for i in range(0,10):
        print(f"Prediction: {outputs[i]}")
        print(f"Target: {targets[i]}")
        print("\n")
    del model, inputs, targets, outputs
    torch.cuda.empty_cache()

# Real Valued Training

In [ ]:
if train_real:
    # Create an instance of the network
    def get_real_model():
        return RealValuedNN(n_conv_layers=2, conv_filters=[8,16], conv_kernel_size=[1,3], n_fc_layers=1, n_fc_units=128)

    # put model on device
    model = get_real_model()
    model.to(device)
    model = torch.compile(model)

    # Print the model architecture
    print(model)
    print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)
    writer = SummaryWriter("Real_valued")
    model_path = rootdir + 'notebooks/real_nn_model.pt'

In [ ]:
# Training loop
if train_real:
    real_train_loss_arr,  real_val_loss_arr = train(model, criterion, optimizer, scheduler, train_loader, test_loader, 100, model_path, writer)
    # save the loss arrays into a single file
    np.savez(rootdir + 'notebooks/real_loss_arrays.npz', train=real_train_loss_arr, val=real_val_loss_arr)
    del model, criterion, optimizer, scheduler, writer
    torch.cuda.empty_cache()
    memsum()

In [ ]:
if test_real:
    # Predictions from Real model
    # Load model
    model = get_real_model()
    model = torch.compile(model)
    
    modelfn = model_path.split('/')[-1]
    modeldir = '/'.join(model_path.split('/')[:-1])
    model_files = os.listdir(modeldir)
    model_files = [f for f in model_files if f.endswith('.pt') and f.startswith(modelfn.split('.')[0])]
    if len(model_files) > 0:
        model_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
        model_path = modeldir + '/' + model_files[-1]
        
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()


In [ ]:
if test_real:
    # Get 10 inputs
    inputs, targets = next(iter(test_loader))
    outputs = model(inputs[0])

    if type(outputs) == torch.Tensor:
        outputs = outputs.cpu().detach().numpy()
    if type(targets) == torch.Tensor:
        targets = targets[0].cpu().detach().numpy()
    for i in range(0,10):
        print(f"Prediction: {outputs[i]}")
        print(f"Target: {targets[i]}")
        print("\n")
    del model, inputs, targets, outputs
    torch.cuda.empty_cache()

In [ ]:
# Load any existing complex and real loss arrays
if 'complex_train_loss_arr' not in locals():
    complex_loss_arrays = np.load(rootdir + 'notebooks/complex_loss_arrays.npz')
    complex_train_loss_arr = complex_loss_arrays['train']
    complex_val_loss_arr = complex_loss_arrays['val']
if 'real_train_loss_arr' not in locals():
    real_loss_arrays = np.load(rootdir + 'notebooks/real_loss_arrays.npz')
    real_train_loss_arr = real_loss_arrays['train']
    real_val_loss_arr = real_loss_arrays['val']

# Plot the loss arrays
plt.figure()
plt.plot(complex_train_loss_arr, label='Complex Train')
plt.plot(complex_val_loss_arr, label='Complex Validation')
plt.plot(real_train_loss_arr, label='Real Train')
plt.plot(real_val_loss_arr, label='Real Validation')
plt.legend()
plt.show()


## Real Valued in the time domain ( DEV/UNTESTED CODE BELOW )

## Recreate the dataset

In [ ]:
# del train_data, test_data, train_loader, test_loader
# torch.cuda.empty_cache()

train_data = GlitchDataset(datadir, ifos, ml_models, glitches, distributions, NTRAIN, NTEST, device, True, 'train', 'real')
test_data = GlitchDataset(datadir, ifos, ml_models, glitches, distributions, NTRAIN, NTEST, device, True, 'test', 'real')
tr_sampler = FastRandomSampler(train_data, 1024)
te_sampler = FastRandomSampler(test_data, 1024)

train_loader = DataLoader(dataset=train_data,sampler = tr_sampler,  drop_last=False)
test_loader = DataLoader(dataset=test_data, sampler = te_sampler, drop_last=False)
# testitem = test_data.__getitem__(0)
# print(testitem[0][1:20], testitem[1])

In [ ]:
from antiglitch import SnippetNormed
trainitem = train_data.__getitem__(np.random.randint(0, NTRAIN))
testitem = test_data.__getitem__(np.random.randint(0, NTEST))

# Get a real snippet
ifo = np.random.choice(ifos)
ml_model = np.random.choice(ml_models)
glitch_num = random.choice(glitches[ifo][ml_model])
glitch_num = glitch_num['num']
# create a SnippetNormed object
snip = SnippetNormed(ifo, ml_model, glitch_num, datadir)
inf = {}
inf['freqs'] = np.linspace(0, 4096, 513)
snip.set_infer(inf)
# get the glitch data in the time domain
actual_item = snip.whts
# scale it by the training scalings
actual_item = (actual_item - train_data.x_arr_mean) / train_data.x_arr_std
# plot them all side by side
plt.figure()
plt.subplot(1,3,1)
plt.plot(trainitem[0].cpu().numpy())
plt.title('Train')
plt.subplot(1,3,2)
plt.plot(testitem[0].cpu().numpy())
plt.title('Test')
plt.subplot(1,3,3)
plt.plot(actual_item)
plt.title('Actual')
plt.show()


In [ ]:
# Real Valued Training
input_dim = 513  # Number of input features
output_dim = 2  # Number of output units (as specified in the dataset)
# reimport the real valued model from cvnn_models
from cvnn_models import RealValuedNNtd

# Create an instance of the network
model = RealValuedNNtd()

# put model on device
model.to(device)

# Print the model architecture
print(model)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")


In [ ]:

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)
writer = SummaryWriter("Real_valued_td")
model_path = rootdir + 'notebooks/real_td_nn_model.pth'


In [ ]:
# Training loop
if train_real_td:
    train(model, criterion, optimizer, scheduler, train_loader, test_loader, 100, model_path, writer)


In [ ]:
(1024*16384)-(65536*256)

In [ ]:

del model, criterion, optimizer, scheduler, writer
torch.cuda.empty_cache()
memsum()


In [ ]:

if test_real_td:
    # Predictions from Real model
    # Load model
    model = RealValuedNN()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()


In [ ]:

if test_real_td:
    # Get 10 inputs
    inputs, targets = next(iter(test_loader))
    outputs = model(inputs[0].real, inputs[0].imag)

    if type(outputs) == torch.Tensor:
        outputs = outputs.cpu().detach().numpy()
    if type(targets) == torch.Tensor:
        targets = targets[0].cpu().detach().numpy()
    for i in range(0,10):
        print(f"Prediction: {outputs[i]}")
        print(f"Target: {targets[i]}")
        print("\n")
del model, inputs, targets, outputs
torch.cuda.empty_cache()